In [1]:
# Import create_engine, MetaData
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer
from sqlalchemy import insert
from sqlalchemy import select
from sqlalchemy import case, cast, Float

<h2>Setup the Engine and MetaData</h2>

In this exercise, your job is to create an engine to the database that will be used in this chapter. Then, you need to initialize its metadata.

In [2]:
# Define an engine to connect to chapter5.sqlite: engine
engine = create_engine('sqlite:///data/chapter5.sqlite')

# Initialize MetaData: metadata
metadata = MetaData()

<h2>Create the Table to the Database</h2>

Having setup the engine and initialized the metadata, you will now define the census table object and then create it in the database using the metadata and engine from the previous exercise. To create it in the database, you will have to use the .create_all() method on the metadata with engine as the argument.

In [3]:
# Build a census table: census
census = Table('census', metadata,
               Column('state', String(30)),
               Column('sex', String(1)),
               Column('age', Integer()),
               Column('pop2000', Integer()),
               Column('pop2008', Integer()))

# Create the table in the database
metadata.create_all(engine)

<h2>Reading the Data from the CSV</h2>

Leverage the Python CSV module from the standard library and load the data into a list of dictionaries.

In [4]:
# Create an empty list: values_list
values_list = []

# Iterate over the rows
for row in csv_reader:
    # Create a dictionary with the values
    data = {'state': row[0], 'sex': row[1], 'age': row[2], 'pop2000': row[3],
            'pop2008': row[4]}
    # Append the dictionary to the values list
    values_list.append(data)


NameError: name 'csv_reader' is not defined

<h2>Load Data from a list into the Table</h2>

Using the multiple insert pattern, in this exercise, you will load the data from values_list into the table.

In [ ]:
# Build insert statement: stmt
stmt = insert(census)

# Use values_list to insert data: results
results = connection.execute(stmt, values_list)

# Print rowcount
print(results.rowcount)

<h2>Build a Query to Determine the Average Age by Population</h2>

In this exercise, you will use the func.sum() and group_by() methods to first determine the average age weighted by the population in 2008, and then group by sex.

As Jason discussed in the video, a weighted average is calculated as the sum of the product of the weights and averages divided by the sum of all the weights.

In [ ]:
# Calculate weighted average age: stmt
stmt = select([census.columns.sex,
               (func.sum(census.columns.pop2008 * census.columns.age) /
                func.sum(census.columns.pop2008)).label('average_age')
               ])

# Group by sex
stmt = stmt.group_by(census.columns.sex)

# Execute the query and store the results: results
results = connection.execute(stmt).fetchall()

# Print the average age by sex
for result in results:
    print(result.sex, result.average_age)

<h2>Build a Query to Determine the Percentage of Population by Gender and State</h2>

In this exercise, you will write a query to determine the percentage of the population in 2000 that comprised of women. You will group this query by state.

In [ ]:
# import case, cast and Float from sqlalchemy
from sqlalchemy import case, cast, Float

# Build a query to calculate the percentage of females in 2000: stmt
stmt = select([census.columns.state,
    (func.sum(
        case([
            (census.columns.sex == 'F', census.columns.pop2000)
        ], else_=0)) /
     cast(func.sum(census.columns.pop2000), Float) * 100).label('percent_female')
])

# Group By state
stmt = stmt.group_by(census.columns.state)

# Execute the query and store the results: results
results = connection.execute(stmt).fetchall()

# Print the percentage
for result in results:
    print(result.state, result.percent_female)


<h2>Build a Query to Determine the Difference by State from the 2000 and 2008 Censuses</h2>

In this final exercise, you will write a query to calculate the states that changed the most in population. You will limit your query to display only the top 10 states.

In [ ]:
# Build query to return state name and population difference from 2008 to 2000
stmt = select([census.columns.state,
     (census.columns.pop2008-census.columns.pop2000).label('pop_change')
])

# Group by State
stmt = stmt.group_by(census.columns.state)

# Order by Population Change
stmt = stmt.order_by(desc('pop_change'))

# Limit to top 10
stmt = stmt.limit(10)

# Use connection to execute the statement and fetch all results
results = connection.execute(stmt).fetchall()

# Print the state and population change for each record
for result in results:
    print('{}:{}'.format(result.state, result.pop_change))